# Dense subgraphs

| Student Name         | Student-ID |
|----------------------|------------|
| Marco Di Francesco   | 100632815  |
| Loreto García Tejada | 100643862  |
| György Bence Józsa   | 100633270  |
| József-Hunor Jánosi  | 100516724  |
| Sara-Jane Bittner    | 100498554  |

_Learning goal: To familiarize oneself with one definition of the density of subgraphs._

Consider a graph $G = (V, E)$. Given a subset of vertices $S \subseteq V$ , we propose the following
definition of density:

$$ D(S) = \frac{\sum_{i \in S} \sum_{j \in S} w(i,j)}{|S|}, $$

where $w(i, j)$ is the weight of the edge between vertices $i$ and $j$, if it exists, and $0$ otherwise,
and $|S|$ is the cardinality of the set $S$. Note that in unweighted graphs, $w(i, j) \in \{0, 1\}$.

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def density(S: nx.Graph):
    """Calculates the density of (sub)graph S"""
    return S.size(weight='weight') / S.number_of_nodes()

a) Propose a greedy algorithm to find a subgraph with good density according to this definition. Explain the main properties of the algorithm, e.g., does the result depend on the order of execution etc.

Proposition:

We start with $S = \emptyset$.
In the first step, we choose the vertex $v_0$ such that the sum of the weights of the edges connecting to $v$ is maximal. We can call it the vertex with the maximal weight. Add that to $S$.
After that, in each step, we choose a vertex $v_i$ out of the neighbours of $S$, such that $D(S \cup v_i)$ is maximal, and add that to $S$. We repeat this step until $D(S) > D(S \cup v_i)$.
We return S.

In [3]:
def dense_subgraph(G: nx.Graph) -> nx.Graph:
    node_degrees = dict(G.degree)
    max_node = max(zip(node_degrees.values(), node_degrees.keys()))[1]
    S = [max_node]
    while True:
        neighbors = {x for n in S for x in G.neighbors(n)} # here be dragons
        max_value = 0
        candidate = -1
        for v in neighbors:
            S2 = S + [v]
            d1 = density(G.subgraph(S))
            d2 = density(G.subgraph(S2))
            if d2 > d1:
                if d2 > max_value:
                    max_value = d2
                    candidate = v
        if candidate == -1:
            return G.subgraph(S)
        S.append(candidate)


b) Load _dolphins.txt_ data set. In this file, the first line contains the number of vertices. Each subsequent line indicates the two endpoints of an undirected edge. Estimate some reasonable lower and upper bounds on $D$.

In [ ]:
G = nx.Graph()
filename = 'dolphins.txt'
with open(filename) as f:
    n = int(next(f)) # first line is number of vertices
    for line in f:
        i, j = [int(x) for x in line.split()]
        G.add_edge(i, j)

num_edges = G.number_of_edges()

# minimum bound
num_nodes = num_edges * 2
min_bound = num_edges / num_nodes

# maximum bound
num_nodes = np.ceil((1 + np.sqrt(1 + 8 * num_edges)) / 2)
max_bound = num_edges / num_nodes

print(f'Minimum boundary of density D of {filename}: {min_bound}')
print(f'Maximum boundary of density D of {filename}: {max_bound}')

Estimation is done only based on the number of edges that the network possesses, not taking into account the nodes and their connectivity.

For the minimum bound we take the worst case scenario when all the edges do not have any common node within them. (E.g., if the network has 2 edges, our worst case would have their edges like: {1 - 2, 3 - 4}). Like so, the number of edges would be equal to 2 * number_of_edges.

As for the maximum bound we assume that the edges make up a fully connected graph and we estimate the number edges based on this assumption.
When a fully connected network has n nodes, the number of edges is n * (n - 1) / 2. Calculating the number of nodes in a backwards manner from this equation we get the following formula: number_of_nodes = (1 + 1 + 8 * number_of_edges) / 2.

In [ ]:
G = nx.Graph()
filename = 'dolphins.txt'
with open(filename) as f:
    n = int(next(f)) # first line is number of vertices
    for line in f:
        i, j = [int(x) for x in line.split()]
        G.add_edge(i, j)

num_edges = G.number_of_edges()

# minimum bound
num_nodes = num_edges * 2
min_bound = num_edges / num_nodes

# maximum bound
num_nodes = np.ceil((1 + np.sqrt(1 + 8 * num_edges)) / 2)
max_bound = num_edges / num_nodes

print(f'Minimum boundary of density D of {filename}: {min_bound}')
print(f'Maximum boundary of density D of {filename}: {max_bound}')

c) Apply your algorithm on the _dolphins.txt_ data. As a result, provide the maximum density you achieved and the vertices of the corresponding subgraph (in ascending order).

In [ ]:
SG = dense_subgraph(G)

sorted_nodes = list(SG.nodes)
sorted_nodes.sort()
print(f'Density of subgraph: {density(SG)}')
print(f'Nodes in subgraph: {sorted_nodes}')

d) Visualize the graph using Gephi ([https://gephi.org/](https://gephi.org/)). Use different coloring for vertices that are within the subgraph found in the previous step.

In [ ]:
with open('dolphins2.csv', 'w') as f:
    f.write('Source,Target,Subgraph\n')
    for i, j in G.edges:
        if (i, j) in SG.edges:
            f.write(f'{i},{j},1\n')
        else:
            f.write(f'{i},{j},0\n')

![output-subgraph.png](output-subgraph.png)

In [7]:
with open('dolphins2.csv', 'w') as f:
    f.write('Source,Target,Subgraph\n')
    for i, j in G.edges:
        if (i, j) in SG.edges:
            f.write(f'{i},{j},1\n')
        else:
            f.write(f'{i},{j},0\n')

![output-subgraph.png](output-subgraph.png)